## Context Augmented OpenAI Agent

In [90]:
import json
from typing import Sequence

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [91]:
try:
    storage_context = StorageContext.from_defaults(persist_dir="./storage/march")
    march_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="./storage/june")
    june_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="./storage/sept")
    sept_index = load_index_from_storage(storage_context)

    index_loaded=True
    
except:
    index_loaded = False

### Download Data

In [92]:
!mkdir -p 'data/10q/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf' -O 'data/10q/uber_10q_march_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf' -O 'data/10q/uber_10q_june_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_sept_2022.pdf' -O 'data/10q/uber_10q_sept_2022.pdf'

--2024-04-30 15:43:07--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1260185 (1.2M) [application/octet-stream]
Saving to: ‘data/10q/uber_10q_march_2022.pdf’

data/10q/uber_10q_m 100%[===================>]   1.20M  --.-KB/s    in 0.1s    

2024-04-30 15:43:08 (9.53 MB/s) - ‘data/10q/uber_10q_march_2022.pdf’ saved [1260185/1260185]

--2024-04-30 15:43:08--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [93]:
# build indexes across the three data sources
if not index_loaded:
    # load data
    march_docs = SimpleDirectoryReader(
        input_files=["./data/10q/uber_10q_march_2022.pdf"]
    ).load_data()
    june_docs = SimpleDirectoryReader(
        input_files=["./data/10q/uber_10q_june_2022.pdf"]
    ).load_data()
    sept_docs = SimpleDirectoryReader(
        input_files=["./data/10q/uber_10q_sept_2022.pdf"]
    ).load_data()

    # build index
    march_index = VectorStoreIndex.from_documents(march_docs)
    june_index = VectorStoreIndex.from_documents(june_docs)
    sept_index = VectorStoreIndex.from_documents(sept_docs)

    # persist index
    march_index.storage_context.persist(persist_dir="./storage/march")
    june_index.storage_context.persist(persist_dir="./storage/june")
    sept_index.storage_context.persist(persist_dir="./storage/sept")

In [94]:
march_engine = march_index.as_query_engine(similarity_top_k=3)
june_engine = june_index.as_query_engine(similarity_top_k=3)
sept_engine = sept_index.as_query_engine(similarity_top_k=3)

In [95]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=march_engine,
        metadata=ToolMetadata(
            name="uber_march_10q",
            description=(
                "Provides information about Uber 10Q filings for March 2022. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=june_engine,
        metadata=ToolMetadata(
            name="uber_june_10q",
            description=(
                "Provides information about Uber financials for June 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=sept_engine,
        metadata=ToolMetadata(
            name="uber_sept_10q",
            description=(
                "Provides information about Uber financials for Sept 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Try Context-Augmented Agent

In [96]:
from llama_index.core import Document
from llama_index.agent.openai_legacy import ContextRetrieverOpenAIAgent

In [97]:
# toy index - stores a list of abbreviations
texts = [
    "Abbreviation: X = Revenue",
    "Abbreviation: Y = Risk Factors",
    "Abbreviation: Z = Costs",
]
docs = [Document(text=t) for t in texts]
context_index = VectorStoreIndex.from_documents(docs)

In [98]:
context_agent = ContextRetrieverOpenAIAgent.from_tools_and_retriever(
    query_engine_tools,
    context_index.as_retriever(similarity_top_k=1),
    verbose=True,
)

In [99]:
response = context_agent.chat('What is the X of March 2022?')

Context information is below.
---------------------
Abbreviation: X = Revenue
---------------------
Given the context information and not prior knowledge, either pick the corresponding tool or answer the function: What is the X of March 2022?

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: uber_march_10q with args: {"input":"What is the Revenue of March 2022?"}
Got output: The Revenue for March 2022 is $6,854 million.

STARTING TURN 2
---------------



In [100]:
print(str(response))

The Revenue for March 2022 is $6,854 million.


In [101]:
context_agent.chat("What is the X and Z in September 2022?")

Context information is below.
---------------------
Abbreviation: Z = Costs
---------------------
Given the context information and not prior knowledge, either pick the corresponding tool or answer the function: What is the X and Z in September 2022?

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: uber_sept_10q with args: {"input": "What is the Revenue of September 2022?"}
Got output: The Revenue for September 2022 is $8,343 million.

=== Calling Function ===
Calling function: uber_sept_10q with args: {"input": "What are the Costs of September 2022?"}
Got output: Costs of September 2022 include Cost of Revenue, exclusive of Depreciation and Amortization, Operations and Support expenses, and Sales and Marketing expenses.

=== Calling Function ===
Calling function: uber_sept_10q with args: {"input": "What is the Revenue and Costs of September 2022?"}
Got output: The Revenue for September 2022 is $8,343 million, and the Costs for September 2022 are as follows:

AgentChatResponse(response='The Revenue for September 2022 is $8,343 million, and the Costs for September 2022 are as follows:\n- Cost of Revenue, exclusive of Depreciation and Amortization: $5,173 million\n- Operations and Support: $617 million\n- Sales and Marketing: $1,153 million', sources=[ToolOutput(content='The Revenue for September 2022 is $8,343 million.', tool_name='uber_sept_10q', raw_input={'input': 'What is the Revenue of September 2022?'}, raw_output=Response(response='The Revenue for September 2022 is $8,343 million.', source_nodes=[NodeWithScore(node=TextNode(id_='b91a641d-8378-47b2-9162-480a49b1b0c2', embedding=None, metadata={'page_label': '42', 'file_name': 'uber_10q_sept_2022.pdf', 'file_path': 'data/10q/uber_10q_sept_2022.pdf', 'file_type': 'application/pdf', 'file_size': 1178622, 'creation_date': '2024-04-30', 'last_modified_date': '2024-04-30'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_acces

### Use Uber 10-Q as context, use Calculator as Tool

In [ ]:
from llama_index.core.tools import BaseTool, FunctionTool

def gross_profit(revenue: int, cost: int) -> str:
    """Calculates the gross profit and gross margin percentage from revenue and cost."""
    profit = revenue - cost
    margin_percentage = (profit / revenue) * 100
    return f"Gross Profit is {profit}, Gross Margin % is {margin_percentage:.2f}"

magic_tool = FunctionTool.from_defaults(fn=gross_profit, name="gross_profit")


In [ ]:
context_agent = ContextRetrieverOpenAIAgent.from_tools_and_retriever(
    [magic_tool], sept_index.as_retriever(similarity_top_k=3), verbose=True)


In [ ]:
response = context_agent.chat("How much was revenue and cost in the quarter?")

In [ ]:
print(response)

In [ ]:
from llama_index.core.tools import BaseTool, FunctionTool

def gross_profit(revenue: int, cost: int) -> str:
    """Calculates the gross profit and gross margin percentage from revenue and cost."""
    profit = revenue - cost
    margin_percentage = (profit / revenue) * 100
    return f"Gross Profit is {profit}, Gross Margin % is {margin_percentage:.2f}"

def ebit(revenue: int, cost: int, operating_expenses: int) -> str:
    """Calculates Earnings Before Interest and Taxes (EBIT) from revenue, cost of goods sold, and operating expenses."""
    ebit_value = revenue - cost - operating_expenses
    return f"EBIT is {ebit_value}"

def eps(net_income: int, shares_outstanding: int) -> str:
    """Calculates Earnings Per Share (EPS) from net income and number of shares outstanding."""
    eps_value = net_income / shares_outstanding
    return f"EPS is {eps_value:.2f}"

# Create FunctionTool objects for each function
gross_profit_tool = FunctionTool.from_defaults(fn=gross_profit, name="gross_profit")
ebit_tool = FunctionTool.from_defaults(fn=ebit, name="ebit")
eps_tool = FunctionTool.from_defaults(fn=eps, name="eps")

# Add the tools to your context agent
context_agent = ContextRetrieverOpenAIAgent.from_tools_and_retriever(
    [gross_profit_tool, ebit_tool, eps_tool], sept_index.as_retriever(similarity_top_k=3), verbose=True
)

# Example query to the context agent
response = context_agent.chat("How much was revenue, cost, operating expenses, net income, and shares outstanding in the quarter?")
print(response)

In [ ]:
from llama_index.core.tools import BaseTool, FunctionTool

# Correctly define all the growth calculation functions
def bookings_growth(current_bookings: int, previous_bookings: int) -> str:
    growth = ((current_bookings - previous_bookings) / previous_bookings) * 100
    return f"Year-over-Year Bookings Growth is {growth:.2f}%"

def revenue_growth(current_revenue: int, previous_revenue: int) -> str:
    growth = ((current_revenue - previous_revenue) / previous_revenue) * 100
    return f"Year-over-Year Revenue Growth is {growth:.2f}%"

def gross_profit_growth(current_gross_profit: int, previous_gross_profit: int) -> str:
    growth = ((current_gross_profit - previous_gross_profit) / previous_gross_profit) * 100
    return f"Year-over-Year Gross Profit Growth is {growth:.2f}%"

def ebit_growth(current_ebit: int, previous_ebit: int) -> str:
    growth = ((current_ebit - previous_ebit) / previous_ebit) * 100
    return f"Year-over-Year EBIT Growth is {growth:.2f}%"

def eps_growth(current_eps: float, previous_eps: float) -> str:
    growth = ((current_eps - previous_eps) / previous_eps) * 100
    return f"Year-over-Year EPS Growth is {growth:.2f}%"

# Create FunctionTool objects for each function
bookings_growth_tool = FunctionTool.from_defaults(fn=bookings_growth, name="bookings_growth")
revenue_growth_tool = FunctionTool.from_defaults(fn=revenue_growth, name="revenue_growth")
gross_profit_growth_tool = FunctionTool.from_defaults(fn=gross_profit_growth, name="gross_profit_growth")
ebit_growth_tool = FunctionTool.from_defaults(fn=ebit_growth, name="ebit_growth")
eps_growth_tool = FunctionTool.from_defaults(fn=eps_growth, name="eps_growth")

# Add the tools to your context agent
context_agent = ContextRetrieverOpenAIAgent.from_tools_and_retriever(
    [bookings_growth_tool,revenue_growth_tool, gross_profit_growth_tool, ebit_growth_tool, eps_growth_tool], 
    sept_index.as_retriever(similarity_top_k=3), 
    verbose=True
)

# Example query to the context agent
response = context_agent.chat("How much was bookigns growth, revenue growth, gross profit growth, EBIT grwoth and EPS growth in the quarter")
print(response)


In [ ]:
from llama_index.core.tools import BaseTool, FunctionTool

# Definitions of the calculation functions
def gross_profit(revenue: int, cost: int) -> str:
    profit = revenue - cost
    return f"Gross Profit is {profit}"

def ebit(revenue: int, cost: int, operating_expenses: int) -> str:
    ebit_value = revenue - cost - operating_expenses
    return f"EBIT is {ebit_value}"

def eps(net_income: int, shares_outstanding: int) -> str:
    eps_value = net_income / shares_outstanding
    return f"EPS is {eps_value:.2f}"

# Definitions for growth calculation functions
def bookings_growth(current_bookings: int, previous_bookings: int) -> str:
    growth = ((current_bookings - previous_bookings) / previous_bookings) * 100
    return f"Year-over-Year Bookings Growth is {growth:.2f}%"

def revenue_growth(current_revenue: int, previous_revenue: int) -> str:
    growth = ((current_revenue - previous_revenue) / previous_revenue) * 100
    return f"Year-over-Year Revenue Growth is {growth:.2f}%"

def gross_profit_growth(current_gross_profit: int, previous_gross_profit: int) -> str:
    growth = ((current_gross_profit - previous_gross_profit) / previous_gross_profit) * 100
    return f"Year-over-Year Gross Profit Growth is {growth:.2f}%"

def ebit_growth(current_ebit: int, previous_ebit: int) -> str:
    growth = ((current_ebit - previous_ebit) / previous_ebit) * 100
    return f"Year-over-Year EBIT Growth is {growth:.2f}%"

def eps_growth(current_eps: float, previous_eps: float) -> str:
    growth = ((current_eps - previous_eps) / previous_eps) * 100
    return f"Year-over-Year EPS Growth is {growth:.2f}%"

# Creation of FunctionTool objects for each function
tools = [
    FunctionTool.from_defaults(fn=gross_profit, name="gross_profit"),
    FunctionTool.from_defaults(fn=ebit, name="ebit"),
    FunctionTool.from_defaults(fn=eps, name="eps"),
    FunctionTool.from_defaults(fn=bookings_growth, name="bookings_growth"),
    FunctionTool.from_defaults(fn=revenue_growth, name="revenue_growth"),
    FunctionTool.from_defaults(fn=gross_profit_growth, name="gross_profit_growth"),
    FunctionTool.from_defaults(fn=ebit_growth, name="ebit_growth"),
    FunctionTool.from_defaults(fn=eps_growth, name="eps_growth")
]

# Adding the tools to your context agent
context_agent = ContextRetrieverOpenAIAgent.from_tools_and_retriever(
    tools, sept_index.as_retriever(similarity_top_k=3), verbose=True
)

# Example query to the context agent
response = context_agent.chat("Please provide the figures for bookings, revenue, gross profit, EBIT, and EPS growth for the quarter.")
print(response)
